# Python FORTRAN interfacing examples
add short description on scope of this notebook

## Software requirements
short description on necessary Python and FORTRAN compiler environment

## How to use this notebook
provide note on kernel restarts in terms of Python object handling and hypertext links on used IPython magics

## Python ctypes example

The ctypes module is part of the operating system services included in the standard Python software environment (both 2.7.x and 3.7.x) and provides methods for shared library function calling and to setup C compatible datatypes. It can therefore be used to wrap externally compiled executables/shared libraries in Python scripts. Basic compatibility with FORTRAN routines is given because functions and subroutines in FORTRAN are implemented in a similar way as in C and because several compiler independent C interoperability features were introduced with the Fortran 2003 standard. Therefore, FORTRAN code translated with any modern compiler should in principle be compatible with the code and workflows presented here. More details on how to assure interoperability, however, are given in the best practices section below. In this section only a minimalistic Jupyter based FORTRAN code development workflow and the basic mechanics of Python/FORTRAN interfacing are shown.

### Shared library building

Using IPython file magics a simple FORTRAN routine is written to disk and compiled as shared library using the gfortran compiler. The FORTRAN routine `sqr1` calculates the square of a given integer (replacing the original value!). A Linux utility can be used to investigate the objects/symbols in the shared library. It contains a function called `sqr1_` which can be accessed via Python using the ctypes module. (Note, that the compiler has changed the subroutine's name, a process called name mangling. Different FORTRAN compilers might have different conventions here, thus requiring manual intervention on the Python/C side of the workflow. For information on how to anticipate this in a convenient way see the best practices section below.)

In [1]:
%%writefile sqr1.f90
subroutine sqr1(a)
    implicit none
    integer :: a
    a = a**2
end subroutine sqr1

Overwriting sqr1.f90


In [2]:
%%script bash
echo `which gfortran` && gfortran --version # optional
gfortran -shared -fpic sqr1.f90 -o sqr1.so

/usr/bin/gfortran
GNU Fortran (Ubuntu 7.4.0-1ubuntu1~18.04.1) 7.4.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
!readelf --symbols sqr1.so


Symbol table '.dynsym' contains 11 entries:
   Num:    Value          Size Type    Bind   Vis      Ndx Name
     0: 0000000000000000     0 NOTYPE  LOCAL  DEFAULT  UND 
     1: 0000000000000000     0 NOTYPE  WEAK   DEFAULT  UND __cxa_finalize
     2: 0000000000000000     0 NOTYPE  WEAK   DEFAULT  UND _ITM_registerTMCloneTable
     3: 0000000000000000     0 NOTYPE  WEAK   DEFAULT  UND _ITM_deregisterTMCloneTab
     4: 0000000000000000     0 NOTYPE  WEAK   DEFAULT  UND __gmon_start__
     5: 0000000000201020     0 NOTYPE  GLOBAL DEFAULT   18 _edata
     6: 0000000000201028     0 NOTYPE  GLOBAL DEFAULT   19 _end
     7: 000000000000057a    28 FUNC    GLOBAL DEFAULT    9 sqr1_
     8: 0000000000000460     0 FUNC    GLOBAL DEFAULT    6 _init
     9: 0000000000201020     0 NOTYPE  GLOBAL DEFAULT   19 __bss_start
    10: 0000000000000598     0 FUNC    GLOBAL DEFAULT   10 _fini

Symbol table '.symtab' contains 48 entries:
   Num:    Value          Size Type    Bind   Vis      Ndx Name
     0: 

### Interfacing with Python ctypes
In Python a shared library can be accessed using the ctypes module. By default arguments in FORTRAN are passed by reference, thus FORTRAN subroutine arguments must be passed as memory addresses, i.e. as pointers to an already existing valid C datatype. The FORTRAN subroutine above expects an integer type as argument. To this end, a pointer object containing a C integer datatype (that is initialized using a Python integer) is constructed. Calling the FORTRAN subroutine with this pointer triggers the calculation of the square value with replacement of the original value in the associated memory block. A conversion back into a Python integer datatype can be achieved using ctypes attributes as shown below.

In [4]:
def print_object_info(pyobject):
    return print(id(pyobject),type(pyobject),pyobject)

In [5]:
import ctypes as ctypes
fortran_subroutine = ctypes.CDLL('./sqr1.so')

In [6]:
python_integer_object = 7 # set an integer value to be squared by the FORTRAN routine
print_object_info(python_integer_object)

94331627975904 <class 'int'> 7


In [7]:
ctypes_integer_object = ctypes.c_int(python_integer_object) # create a C integer type from above Python integer
print_object_info(ctypes_integer_object)

140475436833104 <class 'ctypes.c_int'> c_int(7)


In [8]:
ctypes_pointer_object = ctypes.pointer(ctypes_integer_object) # create a pointer object with the C integer type above
print_object_info(ctypes_pointer_object)

140475436830792 <class '__main__.LP_c_int'> <__main__.LP_c_int object at 0x7fc2fc831048>


In [9]:
_ = fortran_subroutine.sqr1_(ctypes_pointer_object) # the value is calculated in place!
print_object_info(ctypes_pointer_object)

140475436830792 <class '__main__.LP_c_int'> <__main__.LP_c_int object at 0x7fc2fc831048>


In [10]:
print(ctypes_pointer_object.contents)

c_int(49)


In [11]:
print_object_info(ctypes_pointer_object.contents.value)

94331627977248 <class 'int'> 49


### References
* https://docs.python.org/3/library/ctypes.html
* [https://software.intel.com/fortran-compiler-developer-guide](https://software.intel.com/en-us/fortran-compiler-developer-guide-and-reference-mixed-language-programming)

## Numpy multi-dimensional array example
add short example using the numpy ctypeslib and the ctypes attribute of numpy arrays

## Stumbling blocks
cover UNIX environment and FORTRAN compiler stuff, incomplete collection...

## C/FORTRAN interoperability issues
basically,
* https://northstar-www.dartmouth.edu/doc/solaris-forte/manuals/fortran/prog_guide/11_cfort.html
* https://gcc.gnu.org/onlinedocs/gcc-8.2.0/gfortran/Interoperability-with-C.html

## Best practices
cover Python/C/FORTRAN interfacing in terms of best practices for use in ocean model development

## Pytest example
cover best practices towards test-driven development of ocean model FORTRAN subroutines?

## Software environment

In [12]:
!gfortran --version

GNU Fortran (Ubuntu 7.4.0-1ubuntu1~18.04.1) 7.4.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [13]:
!conda --version

conda 4.7.10


In [14]:
!conda list

# packages in environment at /srv/conda/envs/notebook:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main    defaults
alembic                   1.1.0                      py_0    conda-forge
asn1crypto                0.24.0                py37_1003    conda-forge
async_generator           1.10                       py_0    conda-forge
attrs                     19.1.0                     py_0    conda-forge
backcall                  0.1.0                      py_0    conda-forge
bleach                    3.1.0                      py_0    conda-forge
blinker                   1.4                        py_1    conda-forge
bzip2                     1.0.8                h516909a_1    conda-forge
ca-certificates           2019.9.11            hecc5488_0    conda-forge
certifi                   2019.9.11                py37_0    conda-forge
certipy                   0.1.3                      py_0    conda-forge
cff